### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

In [127]:
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria aora 
        self.chegada = tempoChegada

    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    
class FIFO(object):     
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que chegam no tempo zero, no caso de FIFO - todos

    def pronto(self,Processo):
        # implemente aqui o que escalonador faz quando surge um novo processo pronto: add na fila
        self.prontos.append(Processo)
        
    def proximo(self):
        # implemente aqui a politica de escalonamento para definir um processo a ser executado
            # retira da fila o proximo a ser executado
            # "executa" -> somar o tempo do processo ao tempo total de execução        
        p = self.prontos.pop(0)
        return p #processo p eh escolhido
    
class SJF(object):     
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que chegam no tempo zero, no caso do SJF - todos
        
        # realiza a ordenação dos processos, visto que no SJF todos chegam no tempo 0
        for j in range(0, len(self.prontos)):
            for i in range(0, len(self.prontos)-1):
                if(self.prontos[i].tam > self.prontos[i+1].tam):
                    aux = self.prontos[i+1]
                    self.prontos[i+1] = self.prontos[i]
                    self.prontos[i] = aux
        
    def pronto(self,Processo):
        # implemente aqui o que escalonador faz quando surge um novo processo pronto: add na fila
        self.prontos.append(Processo)
        
    def proximo(self):
        # implemente aqui a politica de escalonamento para definir um processo a ser executado
            # os processos já estão ordenados por tempo menor, então é só seguir a fila
            # "executa" -> retira da fila e soma o tempo do processo ao tempo total de execução         
        p = self.prontos.pop(0)
        return p #processo p eh escolhido

Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

In [128]:
nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([10,10,10,10])
tamanho2 = np.array([100,10,10,10])
tempoExecMedio = 0
tempoCheg = [0,0,0,0]

total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,tempoCheg[i])) #cria uma lista procs de Processos
    
procs2 = []
for i in range(nprocs):
    procs2.append(Processo(nomes[i],chanceio[i],tamanho2[i],0,tempoCheg[i])) #cria uma lista procs de Processos    



Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Percebam que na terceira linha é instanciado o escalonador (neste caso, um round_robin). Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

In [129]:
quantum = 2
tempoBloq = 1
tempoMedioExec = 0
tempoMedioResp = 0

escFIFO = FIFO(procs) #troque escalonador pelo seu escalonador
escSJF = SJF(procs2)

bloqueados = []
tempo = 0

random.seed(0)

def tempExec(self,tempoTermino):
    return tempoTermino - self.chegada

# tempo de resposta: quanto tempo demorou para o processo ser executado pela primeira vez desde que foi criado
def tempResp(self,tempoExec1):
    return tempoExec1 - self.chegada

# tempo de espera: quanto tempo passou no estado pronto antes de ser executado pela primeira vez
# def tempEsp()

while total>0:
    p = escFIFO.proximo()
    if(p is not None):
        rodou, _ = p.roda() #adicione quantum como parâmetro, por enquanto nao temos E/S
        
        if(p.tam>0):
            escFIFO.pronto(p)
        total-=rodou
        print("Tempo resposta - ", p.nome,": ", tempResp(p, tempo))
        tempoMedioResp += tempResp(p, tempo)
        tempo+=rodou
        print("Tempo execucao - ", p.nome,": ", tempExec(p, tempo))
        tempoMedioExec += tempExec(p, tempo)
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
print("")
print("Tempo medio execucao: ", tempoMedioExec/nprocs)
print("Tempo medio resposta: ", tempoMedioResp/nprocs)

# ------ FIFO
# A  rodou por  10  timeslice, faltam  0
# Tempo resposta -  A :  0
# Tempo execucao -  A :  10
# B  rodou por  10  timeslice, faltam  0
# Tempo resposta -  B :  10
# Tempo execucao -  B :  20
# C  rodou por  10  timeslice, faltam  0
# Tempo resposta -  C :  20
# Tempo execucao -  C :  30
# D  rodou por  10  timeslice, faltam  0
# Tempo resposta -  D :  30
# Tempo execucao -  D :  40

# Tempo medio execucao:  25.0
# Tempo medio resposta:  15.0

# ------ SJF
# B  rodou por  10  timeslice, faltam  0
# Tempo resposta -  B :  0
# Tempo execucao -  B :  10
# C  rodou por  10  timeslice, faltam  0
# Tempo resposta -  C :  10
# Tempo execucao -  C :  20
# D  rodou por  10  timeslice, faltam  0
# Tempo resposta -  D :  20
# Tempo execucao -  D :  30
# A  rodou por  100  timeslice, faltam  0
# Tempo resposta -  A :  30
# Tempo execucao -  A :  130

# Tempo medio execucao:  47.5
# Tempo medio resposta:  15.0

A  rodou por  10  timeslice, faltam  0
Tempo resposta -  A :  0
Tempo execucao -  A :  10
B  rodou por  10  timeslice, faltam  0
Tempo resposta -  B :  10
Tempo execucao -  B :  20
C  rodou por  10  timeslice, faltam  0
Tempo resposta -  C :  20
Tempo execucao -  C :  30
D  rodou por  10  timeslice, faltam  0
Tempo resposta -  D :  30
Tempo execucao -  D :  40

Tempo medio execucao:  25.0
Tempo medio resposta:  15.0


### Laboratório: ###


1 - Altere o simulador acima para calcular o tempo de execucao medio e tempo de resposta 

2 - Implemente o escalonador por FIFO e SJF e verifique seus tempos de execução e espera.

3 - Faça em outra célula uma implementação do STCF e Round Robin, alterando o p.roda() para receber o quantum


Na segunda feira (15) haverá um questionário no ColabWeb onde você deverá utilizar suas implementações para responder às perguntas. Na terça de manhã farei uma breve conferência para tirar dúvidas.

In [ ]:
class STCF(object):     
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que chegam no tempo zero        
        
    def pronto(self,Processo):
        # implemente aqui o que escalonador faz quando surge um novo processo pronto: add na fila
        for j in range(0, len(self.prontos)):
            for i in range(0, len(self.prontos)-1):
                if(self.prontos[i].tam > self.prontos[i+1].tam):
                    aux = self.prontos[i+1]
                    self.prontos[i+1] = self.prontos[i]
                    self.prontos[i] = aux
        self.prontos.append(Processo)
        
    def proximo(self):
        # implemente aqui a politica de escalonamento para definir um processo a ser executado
            # os processos serão ordenados pelo tempo e depois retirados da fila
            # "executa" -> retira da fila e soma o tempo do processo ao tempo total de execução
        p = self.prontos.pop(0)
        return p #processo p eh escolhido
